## **Entrenamiento del Modelo y Selección de Hiperparámetros**
---

In [2]:
"""
Como el codigo consta de un csv, es suficiente solo con ejectuar los comandos respectivos para cargarlo
a nuestro NoteBook:
"""
#Usamos la opción interactiva para cargar nuestros datos desde nuestro equipo local.
from google.colab import files
uploaded = files.upload()
#Una vez seleccionamos el archivo, usamos pandas e io para leer el archivo csv
import pandas as pd
import io
df = pd.read_csv(io.BytesIO(uploaded['valoraciones_usuarios.csv']), sep = ",")
# Almacenamos los datos en un dataframe y vemos algunos temas descriptivos
df.head()

Saving valoraciones_usuarios.csv to valoraciones_usuarios (1).csv


,ID_CLIENTE,FECHA_INTERACCION,NOMBRE_CORTO_DE_EXPERIENCIA,CALIFICACION
0,1,1/4/2021 9:47,Bono virtual Crepes&Waffles 10,5
1,2,1/7/2021 17:22,Curso Prueba,4
2,3,1/13/2021 17:07,Bono mercado Justo y Bueno 30,5
3,4,1/19/2021 17:30,Bono virtual Crepes&Waffles 10,5
4,5,1/30/2021 13:30,Zapatoca 31,1


In [ ]:
pip install tensorflow_recommenders

In [5]:
import tensorflow_recommenders as tfrs
import tensorflow as tf
import numpy as np

Una vez tenemos nuestros datos cargados, procedemos a definir los datos con el tipo de variable que nos funciona para nuestro modelo:

In [6]:
df = df.astype({'ID_CLIENTE':'string','FECHA_INTERACCION':'string','NOMBRE_CORTO_DE_EXPERIENCIA':'string'})

In [7]:
#revisamos que haya quedado bien los cambios de tipo de variable
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11165 entries, 0 to 11164
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ID_CLIENTE                   11165 non-null  string
 1   FECHA_INTERACCION            11165 non-null  string
 2   NOMBRE_CORTO_DE_EXPERIENCIA  11165 non-null  string
 3   CALIFICACION                 11165 non-null  int64 
dtypes: int64(1), string(3)
memory usage: 349.0 KB


Ahora debemos convertir nuestro df de pandas a un df de tensorflow para poder trabajarlo sin problema, posterior a eso, mapeamos y definimos las columnas que vamos a usar en nuestro modelo, estas son: ID_CLIENTE, NOMBRE_CORTO_DE_EXPERIENCIA y CALIFICACION:

In [8]:
 # convertimos pd.DataFrame a tf.data.Dataset
ds = tf.data.Dataset.from_tensor_slices(
      (dict(df[['ID_CLIENTE','NOMBRE_CORTO_DE_EXPERIENCIA']]), df['CALIFICACION']))

    # convertimos y mapeamos las variables que vamos a usar en nuestro modelo
ds = ds.map(lambda x, y: {
    'ID_CLIENTE' : x['ID_CLIENTE'],
    'NOMBRE_CORTO_DE_EXPERIENCIA' : x['NOMBRE_CORTO_DE_EXPERIENCIA'],
    'CALIFICACION' : y
    })

Definimos los datos de prueba y entrenamiento, para el cado de los recomendadores, se sugiere solo hacer split en entrenamiento y prueba cuando se tienen pocos datos como es nuestro caso (total de 11,165 registros), recordemos que un modelo recomendador y mas cuando usa técnias de deep learning, necesita muchos datos, por ende, decidimos solo dividir en entrenamiento y prueba para dejar mas datos en entrenamiento y enfocar nuestro modelo a un mejor rendimiento, si se tuviesen muchos mas datos, se podría pensar en la opción de sacar un grupo de datos para hacer validación:

In [9]:
tf.random.set_seed(42)
shuffled = ds.shuffle(11165, seed=42, reshuffle_each_iteration=False)
#Definimos datos de entrenamiento y prueba
train = shuffled.take(8932)
test = shuffled.skip(8932).take(2233)

Procedemos a definir los valores unicos de nuestros usuarios y nuestras experiencias que posteriormente se volverán los embeddings respectivos para nuestro modelo:

In [10]:
nombres_experiencias = ds.batch(1_000_000).map(lambda x: x["NOMBRE_CORTO_DE_EXPERIENCIA"])
user_ids = ds.batch(1_000_000).map(lambda x: x["ID_CLIENTE"])
#Sacamos los valores unicos tanto de las experiencias como de los usuarios, estos son los que se usaran como embedings posteriormente
unique_nombres_experiencias = np.unique(np.concatenate(list(nombres_experiencias)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

Definimos la función de nuestro modelo de ranking:

In [11]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Computamos embeddings para usuarios.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Computamos embeddings para experiencias.
    self.experiencias_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_nombres_experiencias, mask_token=None),
      tf.keras.layers.Embedding(len(unique_nombres_experiencias) + 1, embedding_dimension)
    ])

    # Computamos predicciones.
    self.ratings = tf.keras.Sequential([
      # Aprendizaje de multiples capas densas.
      tf.keras.layers.Dense(256, activation="relu"), #capa densa con 256 neuronas y activación Relu
      tf.keras.layers.Dense(64, activation="relu"), #capa densa con 64 neuronas y activación Relu
      # Hacemos las predicciones de los rating en la ultima capa (1 neurona).
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    ID_CLIENTE, NOMBRE_CORTO_DE_EXPERIENCIA = inputs

    user_embedding = self.user_embeddings(ID_CLIENTE)
    experiencia_embedding = self.experiencias_embeddings(NOMBRE_CORTO_DE_EXPERIENCIA)

    return self.ratings(tf.concat([user_embedding, experiencia_embedding], axis=1))

Revisemos rapidamente los primeros resultados de nuestro modelo:

In [12]:
RankingModel()((["42"], ["Bono virtual Crepes&Waffles 10"]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.01952074]], dtype=float32)>

In [13]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

## **Evaluación o Aplicación del modelo**
---

Ahora definamos las metricas y la función de perdida de nuestro modelo:

In [14]:
from typing import Dict, Text

Procedamos a integrar todo en una sola función con el modelo que hemos definido anteriormente:

Como nuestro modelo se basa en un problema de regresión, nuestra salida es de naturaleza continua ya que estamos prediciendo los ratings de todos los usuarios en nuestra matris de valoraciones, procedemos a usar como métrica el RMSE, sin embargo, Keras nos ofrece muchas mas opciones, entre ellas para el caso de regresión métricas como el MAE y MAPE entre muchas otras.

In [15]:
class ExperienciasModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["ID_CLIENTE"], features["NOMBRE_CORTO_DE_EXPERIENCIA"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("CALIFICACION")

    rating_predictions = self(features)

    # El task procesa la perdida y las metricas.
    return self.task(labels=labels, predictions=rating_predictions)

Compilamos con un optimizador Adagrad:

In [16]:
model = ExperienciasModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [17]:
cached_train = train.shuffle(11195).batch(8192).cache()
cached_test = test.batch(4096).cache()

Anteriormente sacamos los datos de entrenamiento y prueba para el fit:

In [18]:
model.fit(cached_train, epochs=1000)

Epoch 1/1000
2/2 [==============================] - 1s 16ms/step - root_mean_squared_error: 4.6825 - loss: 16.1828 - regularization_loss: 0.0000e+00 - total_loss: 16.1828
Epoch 2/1000
2/2 [==============================] - 0s 13ms/step - root_mean_squared_error: 2.8411 - loss: 14.0276 - regularization_loss: 0.0000e+00 - total_loss: 14.0276
Epoch 3/1000
2/2 [==============================] - 0s 13ms/step - root_mean_squared_error: 3.0265 - loss: 4.9265 - regularization_loss: 0.0000e+00 - total_loss: 4.9265
Epoch 4/1000
2/2 [==============================] - 0s 12ms/step - root_mean_squared_error: 1.0672 - loss: 1.3315 - regularization_loss: 0.0000e+00 - total_loss: 1.3315
Epoch 5/1000
2/2 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9744 - loss: 1.0071 - regularization_loss: 0.0000e+00 - total_loss: 1.0071
Epoch 6/1000
2/2 [==============================] - 0s 13ms/step - root_mean_squared_error: 0.9043 - loss: 0.8313 - regularization_loss: 0.0000e+00 - t

In [19]:
model.evaluate(cached_test, return_dict=True)

1/1 [==============================] - 1s 656ms/step - root_mean_squared_error: 0.5221 - loss: 0.2726 - regularization_loss: 0.0000e+00 - total_loss: 0.2726


{'root_mean_squared_error': 0.5221332907676697,
 'loss': 0.2726231515407562,
 'regularization_loss': 0,
 'total_loss': 0.2726231515407562}

Finalmente, vamos a revisar para el usuario 42, cuales son sus experiencias que fueron valoradas en la **realidad**:

In [20]:
df.query("ID_CLIENTE == '42'")

,ID_CLIENTE,FECHA_INTERACCION,NOMBRE_CORTO_DE_EXPERIENCIA,CALIFICACION
99,42,5/20/2021 11:28,Restaurantes 50,5
3671,42,3/14/2022 10:07,Restaurantes 50 IFood,5


Podemos ver que solo calificó 2 experiencias, cada una con 5 respectivamente, una es de un bono de Restaurantes por valor de 50.000 y el otro el mismo pero con la cadena de IFood.

Ahora porcedamos a ver lo que el modelo le predijo a ese usuario en esas mismas experiencias y en otras:

In [21]:
test_ratings = {}
test_experiencias_titles = ["Falabella 50", "Bono virtual Crepes&Waffles 10", "Bono mercado Justo y Bueno 30","Restaurantes 50","Restaurantes 50 IFood","Supermercados 50","Hamburguesa Corral Take Out","Cine Colombia 2D","Combo hamburguesa el Corral","Cineco Combo 2"]
for experiencia_title in test_experiencias_titles:
  test_ratings[experiencia_title] = model({
      "ID_CLIENTE": np.array(["42"]),
      "NOMBRE_CORTO_DE_EXPERIENCIA": np.array([experiencia_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
Bono mercado Justo y Bueno 30: [[5.506077]]
Combo hamburguesa el Corral: [[5.506077]]
Supermercados 50: [[5.500476]]
Bono virtual Crepes&Waffles 10: [[5.4962535]]
Restaurantes 50 IFood: [[5.43136]]
Cineco Combo 2: [[5.414588]]
Restaurantes 50: [[5.4089794]]
Cine Colombia 2D: [[5.295884]]
Falabella 50: [[5.0022907]]
Hamburguesa Corral Take Out: [[3.7388709]]


Como podemos observar, el modelo le asigna unas valoraciones cercanas a las dos experiencias que este usuario había calificado realmente, siendo asi los resultados, el top 5 (ordenado) de recomendaciones para este usuario podrías ser con cierto grado de certeza sobre su afinidad las siguientes:

1. Bono mercado Justo y Bueno 30
2. Combo hamburguesa el Corral
3. Supermercados 50
4. Bono virtual Crepes&Waffles 10
5. Restaurantes 50 IFood

Procedamos a ver un ejemplo similar con otro usuario:




In [22]:
df.query("ID_CLIENTE == '1080'")

,ID_CLIENTE,FECHA_INTERACCION,NOMBRE_CORTO_DE_EXPERIENCIA,CALIFICACION
4485,1080,3/30/2022 17:23,hamburguesa Corral take out,1
4486,1080,3/30/2022 17:23,hamburguesa Corral take out,1
9162,1080,11/16/2022 11:44,Cineco Combo 2,3
9163,1080,11/16/2022 11:44,Cineco Combo 2,3
9164,1080,11/16/2022 11:44,Cineco Combo 2,3
9165,1080,11/16/2022 11:45,Cine Colombia 2D,4
9166,1080,11/16/2022 11:45,Cine Colombia 2D,4
9167,1080,11/16/2022 11:45,Cine Colombia 2D,4
9168,1080,11/16/2022 11:46,Cine Colombia 2D,4
9169,1080,11/16/2022 11:46,Cine Colombia 2D,4


Podemos ver que este usuario ha valorado multiples experiencias en la realidad, siendo las mejor valoradas las que tienen que ver con cine y las peor valoradas las que tienen que ver con hamburguesas del Corral, ahora veamos que nos dice el modelo con sus predicciones de este usuario:

In [23]:
test_ratings = {}
test_experiencias_titles = ["Falabella 50", "Bono virtual Crepes&Waffles 10", "Bono mercado Justo y Bueno 30","Restaurantes 50","Restaurantes 50 IFood","Supermercados 50","Hamburguesa Corral Take Out","Cine Colombia 2D","Combo hamburguesa el Corral","Cineco Combo 2"]
for experiencia_title in test_experiencias_titles:
  test_ratings[experiencia_title] = model({
      "ID_CLIENTE": np.array(["1080"]),
      "NOMBRE_CORTO_DE_EXPERIENCIA": np.array([experiencia_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
Bono virtual Crepes&Waffles 10: [[4.261829]]
Restaurantes 50: [[3.9703622]]
Cine Colombia 2D: [[3.503312]]
Cineco Combo 2: [[3.083671]]
Supermercados 50: [[2.755244]]
Bono mercado Justo y Bueno 30: [[2.4995558]]
Combo hamburguesa el Corral: [[2.4995558]]
Restaurantes 50 IFood: [[1.7339679]]
Falabella 50: [[1.0536253]]
Hamburguesa Corral Take Out: [[0.41820118]]


Como podemos ver, nuevamente el modelo hace un buen trabajo asignando las valoraciones de lo que el usuario valoró realmente, podemos ver que efectivamente el asigna unas buenas valoraciones a lo relacionado con cine y unas malas valoraciones relacionado con el corral, por ende, un top 5 (ordenado) de las recomendaciones para este usuario podrían ser:

1. Bono virtual Crepes&Waffles 10
2. Restaurantes 50
3. Cine Colombia 2D
4. Cineco Combo 2
5. Supermercados 50

Cabe anotar, que con este usuario al igual que el anterior, hemos elegido algunas experiencias como ejemplo para este ejercicio, sin embargo, la idea es coger la de mayor valoraciones de todas las experiencias existentes, otro tema a tener en cuenta es que debemos mostrarle las recomendaciones al usuario en un orden descendente y de izqueirda a derecha (o según como tengamos ordenada nuestro front-page) para que asi podamos mostrar de primera y como más importante, la experiencia que ha tenido mayor valoración en el resultado de la predicción del modelo.

De esta forma vemos como podríamos implementar recomendaciones a todos los usuarios ya que la matriz de valoraciones en este momento se encuentra totalmente llena gracias a nuestro modelo basado en redes neuronales.
